In [1]:
# libraries
import kagglehub
import pandas as pd
import ast
import numpy as np
import gc
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from collections import Counter


# Loading and preprocessing of the movie dataset

In [2]:
# movies dataset
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

# Load the dataset (assuming the main file is 'movies_metadata.csv')
data_path = f"{path}/movies_metadata.csv"
movies_meta = pd.read_csv(data_path)

C:\Users\nickl\AppData\Local\Temp\ipykernel_9968\24608770.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_meta = pd.read_csv(data_path)


In [3]:
movies_meta = movies_meta[(movies_meta["adult"] == 'True') | (movies_meta["adult"] == 'False')]
movies_meta["adult"] = movies_meta["adult"].astype(bool)
movies_meta["budget"] = movies_meta["budget"].astype(int)
movies_meta = movies_meta[movies_meta["id"].str.isnumeric()] # there are some invalid id values. These rows also have other incorrect data types, so we drop them
movies_meta["id"] = movies_meta["id"].astype(int)
movies_meta["original_title"] = movies_meta["original_title"].astype(str)
movies_meta["popularity"] = pd.to_numeric(movies_meta["popularity"], errors='coerce') # some of the popularity entries were strings, converted to float now. This was also a warning upon loading the csv
movies_meta.drop(columns=["belongs_to_collection", "homepage", "tagline", "poster_path"], inplace=True) # too many NaN values. And poster_path is just a filepath to a jpg file

In [4]:
movies_meta['genres'] = movies_meta['genres'].apply(ast.literal_eval)

movies_meta['genre_names'] = movies_meta['genres'].apply(lambda x: [genre['name'] for genre in x])

# one hot encoding for genres
mlb_genres = MultiLabelBinarizer()
genre_encoded = pd.DataFrame(mlb_genres.fit_transform(movies_meta['genre_names']),
                              columns=mlb_genres.classes_,
                              index=movies_meta.index)

# Handle NaN values before applying ast.literal_eval
movies_meta['spoken_languages'] = movies_meta['spoken_languages'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

movies_meta['languages'] = movies_meta['spoken_languages'].apply(lambda x: [lang['name'] for lang in x] if x else [])

# one hot encoding for languages
mlb_languages = MultiLabelBinarizer()
mlb_languages.fit_transform(movies_meta["languages"])
languages_encoded = pd.DataFrame(mlb_languages.fit_transform(movies_meta['languages']),
                              columns=mlb_languages.classes_,
                              index=movies_meta.index)
movies_meta = pd.concat([movies_meta, genre_encoded, languages_encoded], axis=1)
del genre_encoded
del languages_encoded


movies_meta['runtime'] = pd.to_numeric(movies_meta['runtime'], errors='coerce').astype('Int64')
movies_meta['release_date'] = movies_meta['release_date'].str[:4]
movies_meta.rename(columns={"release_date": "year"}, inplace=True)
movies_meta = movies_meta.rename(columns={'runtime': 'duration'})
movies_meta.drop(columns=["adult", "imdb_id", "status", "video", "spoken_languages", "genres", "production_companies", "production_countries", "overview", "genre_names", "languages"], inplace=True)

# Loading and preparing the ratings dataset

In [5]:
ratings_path = f"{path}/ratings.csv"
ratings = pd.read_csv(ratings_path)
ratings.drop(columns=["timestamp"], inplace=True)
ratings.shape

(26024289, 3)

# Merging

In [6]:
# Prepare Training Data
merged_df = pd.merge(ratings, movies_meta,left_on='movieId', right_on='id', how='inner')
merged_df = merged_df.loc[:, (merged_df != 0).any(axis=0)] # remove all columns with only zero values. This can be the case for some genre or language columns
del movies_meta
del ratings
merged_df.drop(columns=["id"], inplace=True) # this is now a duplicate column because of the merging, so we can drop it

# Additional preprocessing

In [7]:
merged_df.drop(columns=[""], inplace=True) # the language one hot encoding resulted in one column with an empty string as column name. We drop it
language_columns = [lang for lang in mlb_languages.classes_ if lang in merged_df.columns]

# remove columns of languages that rarely occur, and instead combine them into a single column other language
infrequent_langs = []
for language in language_columns:
    frequency = Counter(merged_df[language])[1] / len(merged_df)
    if frequency < 0.01:
        infrequent_langs.append(language)

merged_df['Other_language'] = 0
for language in infrequent_langs:
    merged_df['Other_language'] = merged_df['Other_language'] | merged_df[language] # Bitwise OR. If any column has 1, the corresponding row in Other_language will be set to 1
    
merged_df.drop(columns=infrequent_langs, inplace=True)

In [8]:
print(list(merged_df.columns))

['userId', 'movieId', 'rating', 'budget', 'original_language', 'original_title', 'popularity', 'year', 'revenue', 'duration', 'title', 'vote_average', 'vote_count', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western', 'Deutsch', 'English', 'Español', 'Français', 'Italiano', 'Português', 'Pусский', 'suomi', 'svenska', 'العربية', 'हिन्दी', '日本語', '普通话', 'Other_language']


In [10]:
merged_df.dropna(how="any", inplace=True)

# Saving dataframe to file

In [13]:
merged_df.to_parquet("merged_df.parquet")

In [69]:
print(merged_df.head())

   rating  budget original_language          original_title  \
0     1.0       0                fr  Trois couleurs : Rouge   
1     3.5       0                fr  Trois couleurs : Rouge   
2     5.0       0                fr  Trois couleurs : Rouge   
3     5.0       0                fr  Trois couleurs : Rouge   
4     3.0       0                fr  Trois couleurs : Rouge   

                                            overview  popularity  \
0  Red This is the third film from the trilogy by...    7.832755   
1  Red This is the third film from the trilogy by...    7.832755   
2  Red This is the third film from the trilogy by...    7.832755   
3  Red This is the third film from the trilogy by...    7.832755   
4  Red This is the third film from the trilogy by...    7.832755   

                                production_countries  year  revenue  duration  \
0  [{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...  1994      0.0        99   
1  [{'iso_3166_1': 'FR', 'name': 'France'}, {'iso.